In [1]:
import torch

import os
parent_dir = 'D:\\Semester_7\\GraduationProject\\SLR\\Viet-Nam-Sign-Language-UTE-K21-\\data_set\\mediapipe_sequences'
a =[]
lable_decode = []
for lable in os.listdir(parent_dir):
    video_dir = os.path.join(parent_dir, lable)
    for file_name in os.listdir(video_dir):
        lable_decode.append(lable)
        input_file = os.path.join(video_dir, file_name)
        tensor = torch.load(input_file)
        tensor = torch.einsum('xyz->zxy', tensor)
        a.append(tensor)

batch = torch.stack(a)

In [3]:
batch[0].shape

torch.Size([3, 25, 65])

In [4]:
batch[0]

tensor([[[0.6748, 0.6952, 0.7111,  ..., 0.0000, 0.0000, 0.0000],
         [0.6749, 0.6953, 0.7111,  ..., 0.0000, 0.0000, 0.0000],
         [0.6747, 0.6950, 0.7108,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.6000, 0.6274, 0.6386,  ..., 0.0000, 0.0000, 0.0000],
         [0.6350, 0.6496, 0.6556,  ..., 0.0000, 0.0000, 0.0000],
         [0.6483, 0.6605, 0.6667,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.2943, 0.2343, 0.2338,  ..., 0.0000, 0.0000, 0.0000],
         [0.2904, 0.2343, 0.2339,  ..., 0.0000, 0.0000, 0.0000],
         [0.3110, 0.2563, 0.2557,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.3714, 0.3089, 0.3070,  ..., 0.0000, 0.0000, 0.0000],
         [0.3716, 0.3092, 0.3070,  ..., 0.0000, 0.0000, 0.0000],
         [0.3718, 0.3093, 0.3071,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.9879, 0.9652, 0.9654,  ..., 0.0000, 0.0000, 0.0000],
         [0.9880, 0.9640, 0.9653,  ..., 0.0000, 0.0000, 0.0000],
         [0.9882, 0.9641, 0.9656,  ..., 0.0000, 0.0000, 0.

In [7]:
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [8]:
import torch.nn.functional as F

# Danh sách cần one-hot encode
lst = lable_decode

# Tạo từ điển ánh xạ chuỗi sang số nguyên
unique_labels = list(set(lst))  # Lấy các giá trị duy nhất
label_to_index = {label: index for index, label in enumerate(unique_labels)}

# Chuyển đổi danh sách chuỗi thành danh sách chỉ số
indices = [label_to_index[label] for label in lst]

# Chuyển danh sách chỉ số thành tensor
tensor_indices = torch.tensor(indices)


In [9]:
tensor_indices

tensor([16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10,  0,  0,  0,  0,  0,  0, 11, 11, 11, 11, 11, 11,  5,  5,  5,  5,
         5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  4,  4,
         4,  4,  4,  4, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
        13, 13, 13, 13, 13, 13, 15, 15, 15, 15, 15, 15,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2, 14, 14, 14, 14, 14, 14,  8,
         8,  8,  8,  8,  8, 12, 12, 12, 12, 12, 12, 19, 19, 19, 19, 19, 19, 19,
        19, 19, 19,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
         3,  3,  3,  3,  3,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  6,  6,  6,  6,  6,  6, 18, 18, 18, 18, 18,
        18, 18, 18, 18, 18, 17, 17, 17, 17, 17, 17,  7,  7,  7,  7,  7,  7])

In [10]:
len(tensor_indices)

197

In [14]:
from torch.utils.data import random_split
batch_size=5
num = len(lable_decode)
train_ratio = 0.8
train_size = int(train_ratio * num)
val_size = num - train_size
dataset = TensorDataset(batch, tensor_indices)
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


In [17]:
for data, labels in train_loader:
    print("data", data.shape)
    print("labels", labels)

data torch.Size([5, 3, 25, 65])
labels tensor([ 0, 13, 10, 13,  2])
data torch.Size([5, 3, 25, 65])
labels tensor([ 4,  1, 14,  3,  9])
data torch.Size([5, 3, 25, 65])
labels tensor([ 7,  3, 12, 16,  3])
data torch.Size([5, 3, 25, 65])
labels tensor([10, 18, 13, 16, 15])
data torch.Size([5, 3, 25, 65])
labels tensor([ 9, 19,  5,  2,  6])
data torch.Size([5, 3, 25, 65])
labels tensor([ 7,  5,  3,  5, 13])
data torch.Size([5, 3, 25, 65])
labels tensor([ 0, 19, 13, 10,  1])
data torch.Size([5, 3, 25, 65])
labels tensor([ 2,  8, 17, 19,  1])
data torch.Size([5, 3, 25, 65])
labels tensor([ 3, 16, 16, 15,  6])
data torch.Size([5, 3, 25, 65])
labels tensor([ 5, 18, 12, 16,  4])
data torch.Size([5, 3, 25, 65])
labels tensor([ 7,  5,  9, 18, 18])
data torch.Size([5, 3, 25, 65])
labels tensor([19, 10,  8,  1, 17])
data torch.Size([5, 3, 25, 65])
labels tensor([ 2, 16, 13,  3,  4])
data torch.Size([5, 3, 25, 65])
labels tensor([ 2,  3, 12,  1, 19])
data torch.Size([5, 3, 25, 65])
labels tensor([ 

In [31]:
# Kết nối khớp cơ thể (Pose Estimation)
body_connections = [
    (1,4),  
    (0,2),  
    (1,3), 
    (2,7),
    (0,5), 
    (6,4),
    (8,5), 
    (3), 
    (6),
    (10),
    (9),
    (12,13),
    (11,14),
    (11,15),
    (12,16),
    (13,17,21),
    (14,18,22),
    (15,19),
    (16,20),
    (15,17),
    (16,18),
    (15,19),
    (16),
    (24,11),
    (12,23),
    (26),  
    (25, 27),  
    (26, 28),  
    (27, 29),  
    (28),  
    (25, 31, 34),  
    (30, 32),  
    (31, 33),  
    (32),  
    (30, 38), 
    (34, 36),
    (35, 37),
    (36), 
    (34, 42, 39),
    (38, 40),
    (39, 41),
    (40),
    (38, 43),
    (42, 44),
    (43, 45),
    (44),
    (47),  
    (46, 48),  
    (47, 49),  
    (48, 50),  
    (49),  
    (46, 52, 55),  
    (51, 53),  
    (52, 54),  
    (53),  
    (51, 59), 
    (55, 57),
    (56, 58),
    (57), 
    (55, 63, 60),
    (59, 61),
    (60, 62),
    (61),
    (59, 64),
    (63, 65),
    (64, 66),
    (65)
]





In [32]:
len(body_connections)

67

In [33]:
# Tạo danh sách chi tiết hơn theo dạng (index, value)
detailed_body_connections = [(i, val) for i, conn in enumerate(body_connections) for val in (conn if isinstance(conn, tuple) else (conn,))]

print(detailed_body_connections)

[(0, 1), (0, 4), (1, 0), (1, 2), (2, 1), (2, 3), (3, 2), (3, 7), (4, 0), (4, 5), (5, 6), (5, 4), (6, 8), (6, 5), (7, 3), (8, 6), (9, 10), (10, 9), (11, 12), (11, 13), (12, 11), (12, 14), (13, 11), (13, 15), (14, 12), (14, 16), (15, 13), (15, 17), (15, 21), (16, 14), (16, 18), (16, 22), (17, 15), (17, 19), (18, 16), (18, 20), (19, 15), (19, 17), (20, 16), (20, 18), (21, 15), (21, 19), (22, 16), (23, 24), (23, 11), (24, 12), (24, 23), (25, 26), (26, 25), (26, 27), (27, 26), (27, 28), (28, 27), (28, 29), (29, 28), (30, 25), (30, 31), (30, 34), (31, 30), (31, 32), (32, 31), (32, 33), (33, 32), (34, 30), (34, 38), (35, 34), (35, 36), (36, 35), (36, 37), (37, 36), (38, 34), (38, 42), (38, 39), (39, 38), (39, 40), (40, 39), (40, 41), (41, 40), (42, 38), (42, 43), (43, 42), (43, 44), (44, 43), (44, 45), (45, 44), (46, 47), (47, 46), (47, 48), (48, 47), (48, 49), (49, 48), (49, 50), (50, 49), (51, 46), (51, 52), (51, 55), (52, 51), (52, 53), (53, 52), (53, 54), (54, 53), (55, 51), (55, 59), (56

In [34]:
detailed_body_connections = [(0, 1), (0, 4), (1, 0), (1, 2), (2, 1), (2, 3), (3, 2), (3, 7), (4, 0), (4, 5), (5, 6), (5, 4), (6, 8), (6, 5), (7, 3), (8, 6), (9, 10), (10, 9), (11, 12), (11, 13), (12, 11), (12, 14), (13, 11), (13, 15), (14, 12), (14, 16), (15, 13), (15, 17), (15, 21), (16, 14), (16, 18), (16, 22), (17, 15), (17, 19), (18, 16), (18, 20), (19, 15), (19, 17), (20, 16), (20, 18), (21, 15), (21, 19), (22, 16), (23, 24), (23, 11), (24, 12), (24, 23), (25, 26), (26, 25), (26, 27), (27, 26), (27, 28), (28, 27), (28, 29), (29, 28), (30, 25), (30, 31), (30, 34), (31, 30), (31, 32), (32, 31), (32, 33), (33, 32), (34, 30), (34, 38), (35, 34), (35, 36), (36, 35), (36, 37), (37, 36), (38, 34), (38, 42), (38, 39), (39, 38), (39, 40), (40, 39), (40, 41), (41, 40), (42, 38), (42, 43), (43, 42), (43, 44), (44, 43), (44, 45), (45, 44), (46, 47), (47, 46), (47, 48), (48, 47), (48, 49), (49, 48), (49, 50), (50, 49), (51, 46), (51, 52), (51, 55), (52, 51), (52, 53), (53, 52), (53, 54), (54, 53), (55, 51), (55, 59), (56, 55), (56, 57), (57, 56), (57, 58), (58, 57), (59, 55), (59, 63), (59, 60), (60, 59), (60, 61), (61, 60), (61, 62), (62, 61), (63, 59), (63, 64), (64, 63), (64, 65), (65, 64), (65, 66), (66, 65)]

In [ ]:
from STGCN import Model
model = Model(in_channels=3, num_nodes=65, inward_edges=body_connections, n_classes=20)

In [18]:
import torch
import torch.nn as nn

In [19]:
batch.shape
len(lable_decode)
tensortest = batch[1]
tensor_reshaped = tensortest.unsqueeze(0)
tensor_reshaped.shape

torch.Size([1, 3, 25, 65])

In [20]:
results = model(tensor_reshaped)
print(model.parameters())
print(results.shape)
# Giả sử model là instance của mô hình của bạn
for name, param in model.named_parameters():
    print(f"Parameter name: {name}")
    print(f"Shape: {param.shape}\n")


<generator object Module.parameters at 0x000001D2E1A725E0>
torch.Size([1, 20])
Parameter name: data_bn.weight
Shape: torch.Size([195])

Parameter name: data_bn.bias
Shape: torch.Size([195])

Parameter name: st_gcn_networks.0.gcn.conv.weight
Shape: torch.Size([192, 3, 1, 1])

Parameter name: st_gcn_networks.0.gcn.conv.bias
Shape: torch.Size([192])

Parameter name: st_gcn_networks.0.tcn.0.weight
Shape: torch.Size([64])

Parameter name: st_gcn_networks.0.tcn.0.bias
Shape: torch.Size([64])

Parameter name: st_gcn_networks.0.tcn.2.weight
Shape: torch.Size([64, 64, 9, 1])

Parameter name: st_gcn_networks.0.tcn.2.bias
Shape: torch.Size([64])

Parameter name: st_gcn_networks.0.tcn.3.weight
Shape: torch.Size([64])

Parameter name: st_gcn_networks.0.tcn.3.bias
Shape: torch.Size([64])

Parameter name: st_gcn_networks.1.gcn.conv.weight
Shape: torch.Size([192, 64, 1, 1])

Parameter name: st_gcn_networks.1.gcn.conv.bias
Shape: torch.Size([192])

Parameter name: st_gcn_networks.1.tcn.0.weight
Shape: 

In [35]:
optimizer = optim.SGD(model.parameters(), lr=0.01)
num_epochs = 5
criterion = nn.CrossEntropyLoss()
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs, labels

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass và cập nhật trọng số
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # In ra thông tin sau mỗi 100 batch
        running_loss += loss.item()
        if i % 4 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{i+1}], Loss: {running_loss/100:.4f}')
            running_loss = 0.0

print('Hoàn thành huấn luyện!')

Epoch [1/5], Batch [1], Loss: 0.0520
Epoch [1/5], Batch [5], Loss: 0.1645
Epoch [1/5], Batch [9], Loss: 0.1304
Epoch [1/5], Batch [13], Loss: 0.1420
Epoch [1/5], Batch [17], Loss: 0.1194
Epoch [1/5], Batch [21], Loss: 0.1201
Epoch [1/5], Batch [25], Loss: 0.1066
Epoch [1/5], Batch [29], Loss: 0.1188
Epoch [2/5], Batch [1], Loss: 0.0342
Epoch [2/5], Batch [5], Loss: 0.1189
Epoch [2/5], Batch [9], Loss: 0.0972
Epoch [2/5], Batch [13], Loss: 0.1128
Epoch [2/5], Batch [17], Loss: 0.1085
Epoch [2/5], Batch [21], Loss: 0.0909


KeyboardInterrupt: 

In [32]:
for data, labels in val_dataset:
        print("data", data)
        print("labels", labels)

data tensor([[[0.5361, 0.5496, 0.5643,  ..., 0.7757, 0.7792, 0.7798],
         [0.5369, 0.5501, 0.5654,  ..., 0.7772, 0.7803, 0.7794],
         [0.5375, 0.5502, 0.5658,  ..., 0.7408, 0.7445, 0.7445],
         ...,
         [0.5314, 0.5492, 0.5599,  ..., 0.5908, 0.5888, 0.5875],
         [0.5315, 0.5491, 0.5598,  ..., 0.5881, 0.5866, 0.5863],
         [0.5315, 0.5488, 0.5595,  ..., 0.5891, 0.5866, 0.5849]],

        [[0.2691, 0.2323, 0.2351,  ..., 0.5080, 0.4810, 0.4592],
         [0.2707, 0.2342, 0.2368,  ..., 0.5123, 0.4859, 0.4635],
         [0.2707, 0.2343, 0.2368,  ..., 0.4536, 0.4319, 0.4129],
         ...,
         [0.2936, 0.2527, 0.2556,  ..., 0.4287, 0.4037, 0.3818],
         [0.2937, 0.2527, 0.2556,  ..., 0.4203, 0.3980, 0.3787],
         [0.2937, 0.2527, 0.2556,  ..., 0.4244, 0.3991, 0.3769]],

        [[0.9982, 0.9950, 0.9945,  ..., 0.0000, 0.0000, 0.0000],
         [0.9982, 0.9947, 0.9946,  ..., 0.0000, 0.0000, 0.0000],
         [0.9983, 0.9950, 0.9949,  ..., 0.0000, 0.000

In [33]:
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Giả sử bạn đã có mô hình đã huấn luyện và test_loader là DataLoader cho tập test
# Chúng ta sẽ tạo danh sách để lưu trữ các nhãn dự đoán và nhãn thực tế
all_preds = []
all_labels = []

# Đánh giá mô hình trên tập test
with torch.no_grad():
    model.eval()  # Đặt mô hình ở chế độ đánh giá
    for data, labels in val_dataset:
        data = data.unsqueeze(0)
        outputs = model(data)  # Dự đoán từ mô hình
        predicted_classes = torch.argmax(outputs, dim=1)  # Lấy nhãn có xác suất cao nhất
        print(predicted_classes)
        print("nahn",labels)
        # all_preds.extend(predicted_classes.numpy())  # Thêm nhãn dự đoán vào danh sách
        # all_labels.extend(labels.numpy())  # Thêm nhãn thực tế vào danh sách

# # Tính toán các chỉ số đánh giá
# accuracy = accuracy_score(all_labels, all_preds)
# precision = precision_score(all_labels, all_preds, average='weighted')  # Tính precision cho đa lớp
# recall = recall_score(all_labels, all_preds, average='weighted')  # Tính recall cho đa lớp
# f1 = f1_score(all_labels, all_preds, average='weighted')  # Tính F1-score cho đa lớp
# confusion = confusion_matrix(all_labels, all_preds)  # Tính confusion matrix

# # In ra các kết quả đánh giá
# print(f'Accuracy: {accuracy * 100:.2f}%')
# print(f'Precision: {precision:.2f}')
# print(f'Recall: {recall:.2f}')
# print(f'F1 Score: {f1:.2f}')
# print('Confusion Matrix:')
# print(confusion)


tensor([5])
nahn tensor(12)
tensor([18])
nahn tensor(18)
tensor([14])
nahn tensor(10)
tensor([18])
nahn tensor(18)
tensor([12])
nahn tensor(12)
tensor([19])
nahn tensor(13)
tensor([15])
nahn tensor(15)
tensor([3])
nahn tensor(4)
tensor([15])
nahn tensor(17)
tensor([13])
nahn tensor(13)
tensor([5])
nahn tensor(7)
tensor([12])
nahn tensor(12)
tensor([5])
nahn tensor(18)
tensor([8])
nahn tensor(7)
tensor([18])
nahn tensor(18)
tensor([8])
nahn tensor(8)
tensor([6])
nahn tensor(3)
tensor([18])
nahn tensor(17)
tensor([6])
nahn tensor(6)
tensor([8])
nahn tensor(8)
tensor([8])
nahn tensor(19)
tensor([15])
nahn tensor(15)
tensor([19])
nahn tensor(19)
tensor([12])
nahn tensor(2)
tensor([18])
nahn tensor(18)
tensor([18])
nahn tensor(18)
tensor([14])
nahn tensor(10)
tensor([13])
nahn tensor(13)
tensor([5])
nahn tensor(1)
tensor([19])
nahn tensor(19)
tensor([13])
nahn tensor(14)
tensor([3])
nahn tensor(16)
tensor([5])
nahn tensor(5)
tensor([19])
nahn tensor(19)
tensor([14])
nahn tensor(10)
tensor([

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

# 1. Thiết lập các siêu tham số
batch_size = 64
learning_rate = 0.001
num_epochs = 5

# 2. Chuẩn bị dữ liệu MNIST
transform = transforms.Compose([
    transforms.ToTensor(),  # Chuyển đổi hình ảnh thành tensor
    transforms.Normalize((0.5,), (0.5,))  # Chuẩn hóa
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)



In [19]:
i = 1
for data, labels in train_loader:
    print("data", data.shape)
    print("labels", labels)
    if i ==2:
        break
    
    i+=1

data torch.Size([5, 3, 25, 65])
labels tensor([13, 16,  2,  5,  3])
data torch.Size([5, 3, 25, 65])
labels tensor([13,  5,  7,  2, 18])


In [3]:
outputs = torch.rand(1,3)
labels = torch.tensor([1])
criterion = nn.CrossEntropyLoss()
loss = criterion(outputs, labels)
loss

tensor(1.2292)